In [1]:
import pandas as pd
import numpy as np
import sys
import os
import bs4
from IPython.display import display_html,clear_output, HTML
import re
from datetime import datetime
import ast
import itertools
from tqdm import tqdm,trange
from file_tools import *
from request_tools import *
from parse_tools import *
import load_tools as lt
import pyarrow as pa

pd.set_option('display.max_columns', None)
pd.set_option('display.min_rows', 30)
pd.set_option('compute.use_numexpr', False)

import warnings
from pandas.errors import PerformanceWarning
from sklearn.exceptions import DataConversionWarning, ConvergenceWarning
pd.options.mode.use_inf_as_na = True
warnings.filterwarnings('ignore', category=PerformanceWarning)
warnings.filterwarnings('ignore', category=DataConversionWarning)
warnings.filterwarnings('ignore', category=ConvergenceWarning)


In [9]:
DATASET_DIR = './07-data-compiled/dataset-ss0110-mnmx-7fg'
# Load the dataset
FEATURES,LABELS = lt.load_dataset(DATASET_DIR)
FEATURES = FEATURES.dropna()
FEATURES = FEATURES.T.drop_duplicates().T
LABELS = LABELS.loc[FEATURES.index]
LABELS_WL = LABELS['W/L'].to_frame()
LABELS_CONT = LABELS.drop('W/L',axis=1)


In [10]:
LABELS

W/L   Tm  Opp  Pts_Total   
                       index Team_id                                          
/leagues/NBA_2023.html 8     /teams/ATL/2023.html    1  124  121        245  \
                             /teams/BOS/2023.html    1  133  118        251   
                             /teams/BRK/2023.html    1  128   86        214   
                             /teams/CHO/2023.html    0   99  130        229   
                             /teams/CLE/2023.html    1  114  100        214   
                             /teams/DAL/2023.html    1   96   94        190   
                             /teams/DEN/2023.html    1  126  101        227   
                             /teams/GSW/2023.html    0  129  130        259   
                             /teams/IND/2023.html    1  101   99        200   
                             /teams/LAC/2023.html    1  113  106        219   
                             /teams/LAL/2023.html    0  100  114        214   
                             /teams/MEM/2023.html    1  130   99        229   
                             /teams/MIL/2023.html    1  108   94        202   
                             /teams/NOP/2023.html    0  121  124        245   
                             /teams/NYK/2023.html    0  118  133        251   
...                                                ...  ...  ...        ...   
/leagues/NBA_2015.html 81    /teams/MIA/2015.html    1  105  101        206   
                             /teams/MIL/2015.html    0  100  105        205   
                             /teams/MIN/2015.html    0  113  138        251   
                             /teams/NOP/2015.html    1  108  103        211   
                             /teams/NYK/2015.html    0   90  112        202   
                             /teams/OKC/2015.html    1  138  113        251   
                             /teams/ORL/2015.html    0   88  101        189   
                             /teams/PHI/2015.html    0  101  105        206   
                             /teams/PHO/2015.html    0  101  112        213   
                             /teams/POR/2015.html    0   98  114        212   
                             /teams/SAC/2015.html    1  122   99        221   
                             /teams/SAS/2015.html    0  103  108        211   
                             /teams/TOR/2015.html    1   92   87        179   
                             /teams/UTA/2015.html    0   91  117        208   
                             /teams/WAS/2015.html    0  108  113        221   

                                                   Pts_Spread  
                       index Team_id                           
/leagues/NBA_2023.html 8     /teams/ATL/2023.html           3  
                             /teams/BOS/2023.html          15  
                             /teams/BRK/2023.html          42  
                             /teams/CHO/2023.html         -31  
                             /teams/CLE/2023.html          14  
                             /teams/DAL/2023.html           2  
                             /teams/DEN/2023.html          25  
                             /teams/GSW/2023.html          -1  
                             /teams/IND/2023.html           2  
                             /teams/LAC/2023.html           7  
                             /teams/LAL/2023.html         -14  
                             /teams/MEM/2023.html          31  
                             /teams/MIL/2023.html          14  
                             /teams/NOP/2023.html          -3  
                             /teams/NYK/2023.html         -15  
...                                                       ...  
/leagues/NBA_2015.html 81    /teams/MIA/2015.html           4  
                             /teams/MIL/2015.html          -5  
                             /teams/MIN/2015.html         -25  
                             /teams/NOP/2015.html           5  
                             /teams

In [4]:
FEATURES

Team_facts_venue_rest_days_tgl_basic   
                                                                            Match Info   
                                                                                   H/A   
                       index Team_id                                                     
/leagues/NBA_2023.html 8     /teams/ATL/2023.html                                  1.0  \
                             /teams/BOS/2023.html                                  0.0   
                             /teams/BRK/2023.html                                  0.0   
                             /teams/CHO/2023.html                                  0.0   
                             /teams/CLE/2023.html                                  0.0   
                             /teams/DAL/2023.html                                  1.0   
                             /teams/DEN/2023.html                                  1.0   
                             /teams/GSW/2023.html                                  0.0   
                             /teams/IND/2023.html                                  1.0   
                             /teams/LAC/2023.html                                  0.0   
                             /teams/LAL/2023.html                                  1.0   
                             /teams/MEM/2023.html                                  1.0   
                             /teams/MIL/2023.html                                  1.0   
                             /teams/NOP/2023.html                                  0.0   
                             /teams/NYK/2023.html                                  1.0   
...                                                                                ...   
/leagues/NBA_2015.html 81    /teams/MIA/2015.html                                  0.0   
                             /teams/MIL/2015.html                                  1.0   
                             /teams/MIN/2015.html                                  1.0   
                             /teams/NOP/2015.html                                  1.0   
                             /teams/NYK/2015.html                                  1.0   
                             /teams/OKC/2015.html                                  0.0   
                             /teams/ORL/2015.html                                  0.0   
                             /teams/PHI/2015.html                                  1.0   
                             /teams/PHO/2015.html                                  1.0   
                             /teams/POR/2015.html                                  0.0   
                             /teams/SAC/2015.html                                  0.0   
                             /teams/SAS/2015.html                                  0.0   
                             /teams/TOR/2015.html                                  1.0   
                             /teams/UTA/2015.html                                  0.0   
                             /teams/WAS/2015.html                                  0.0   

                                                                                
                                                                                
                                                  Venue_Strk_Cnt Days_Btw_2GM   
                       index Team_id                                            
/leagues/NBA_2023.html 8     /teams/ATL/2023.html            0.0          3.0  \
                             /teams/BOS/2023.html            0.0          1.0   
                             /teams/BRK/2023.html            0.0          3.0   
                             /teams/CHO/2023.html            1.0          2.0   
                             /teams/CLE/2023.html            1.0          2.0   
                             /teams/DAL/2023.html            4.0          3.0   
                             /teams/DEN/2023.html            0.0          2.0   
        

In [11]:
# get the feature correlation with label
# corr = FEATURES.corrwith(LABELS['Pts_Spread'], axis=0, method='pearson')
# corr = corr.loc[corr.abs().sort_values(ascending=False).index]
# corr.head(30).plot.bar(figsize=(20,5))

In [14]:
import pandas as pd
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif, RFE
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import pearsonr
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold

def __corr_pearsonr__(X,y):
   corr = X.corrwith(y, axis=0, method='pearson').T
   return corr

def __corr_f_classif__(X,y):
   anova_scores = f_classif(X, y)
   return pd.Series(anova_scores[0], index=X.columns)

def __corr_mutual_info_classif__(X,y):
   mutual_info_scores = mutual_info_classif(X, y)
   return pd.Series(mutual_info_scores, index=X.columns)

def __corr_RFE__(X,y):
   rfe = RFE(estimator=LogisticRegression(), n_features_to_select=1)
   rfe.fit(X, y)
   rfe_rank = rfe.ranking_
   return pd.Series(rfe_rank, index=X.columns)

def __corr_Lasso__(X,y):
   lasso = Lasso(alpha=0.1)
   lasso.fit(X, y)
   lasso_coef = lasso.coef_
   return pd.Series(lasso_coef, index=X.columns)

def __corr_RandomForestClassifier__(X,y):
   rf = RandomForestClassifier()
   rf.fit(X, y)
   rf_importances = rf.feature_importances_
   return pd.Series(rf_importances, index=X.columns)


def __corr_PCA__(X,y):
   pca = PCA(n_components=1)
   X_scaled = StandardScaler().fit_transform(X)
   pca.fit(X_scaled, y)
   pca_importances = pca.components_
   return pd.Series(pca_importances, index=X.columns)

def __corr_VarianceThreshold__(X,y=None):
   sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
   sel.fit_transform(X)
   sel_importances = sel.variances_
   return pd.Series(sel_importances, index=X.columns)

METHODS_DICT = {
      'Variance Threshold': __corr_VarianceThreshold__,
      'Pearson': __corr_pearsonr__,
      'ANOVA': __corr_f_classif__,
      'Mutual Information': __corr_mutual_info_classif__,
      # 'RFE': __corr_RFE__,
      'Lasso': __corr_Lasso__,
      'Random Forest': __corr_RandomForestClassifier__,
      # 'PCA': __corr_PCA__,
}

CORR_DF_DICT = {}
TQDM_METHODS_DICT_ITEMS = tqdm(METHODS_DICT.items(),ncols=100)
for METHOD_FUNC_NAME, METHOD_FUNC in TQDM_METHODS_DICT_ITEMS:
   TQDM_METHODS_DICT_ITEMS.set_description(f'Processing {METHOD_FUNC_NAME}')
   METHD_CORR_DF_LIST = []
   for label in LABELS.columns:
      X = FEATURES.iloc[:1000]
      y = LABELS.iloc[:1000][label]
      corr = METHOD_FUNC(X,y) #X.corrwith(y, axis=0, method='pearson').T
      METHD_CORR_DF_LIST.append(corr)
   METHD_CORR_DF = pd.concat(METHD_CORR_DF_LIST, axis=1, keys = LABELS.columns)
   CORR_DF_DICT[METHOD_FUNC_NAME] = METHD_CORR_DF

CORR_DF = pd.concat(CORR_DF_DICT.values(),keys = CORR_DF_DICT.keys(), axis=1)
CORR_DF


Processing RFE:  57%|████████████████████████████                     | 4/7 [00:41<00:36, 12.04s/it]/Users/jasetran/Jase/UM/Git/basketball-analytics/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/jasetran/Jase/UM/Git/basketball-analytics/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/p

KeyboardInterrupt: 

In [7]:


# corr_coef = X.apply(lambda x: pearsonr(x, y)[0])

# # Chi-Squared / ANOVA
# # chi_scores = chi2(X, y)
# anova_scores = f_classif(X, y)

# # Recursive Feature Elimination
# estimator = LogisticRegression()  # or another classifier
# selector = RFE(estimator, n_features_to_select=10)
# selector = selector.fit(X, y)
# rfe_scores = selector.ranking_

# # LASSO
# lasso = Lasso(alpha=0.1)
# lasso.fit(X, y)
# lasso_scores = lasso.coef_

# # Random Forest
# rf = RandomForestClassifier()
# rf.fit(X, y)
# rf_scores = rf.feature_importances_

# # Combine scores into a dataframe
# feature_scores = pd.DataFrame({'Feature': X.columns, 
#                                'Correlation': corr_coef, 
#                             #    'Chi-Squared': chi_scores[0], 
#                                'ANOVA F-Value': anova_scores[0], 
#                                'RFE Ranking': rfe_scores, 
#                                'LASSO': lasso_scores,
#                                'Random Forest': rf_scores})

# # Sort by each method
# sorted_by_corr = feature_scores.sort_values('Correlation', ascending=False)
# # Repeat for other methods
